In [1]:
# !pip install poe-api
!pip install -U poe-api-wrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 5.7 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=28f352d74efb4f25e0af0f648c1798fbe97688a8617b28c49c41f494170b7e1f
  Stored in dir

In [2]:
from poe_api_wrapper import PoeApi
# client = PoeApi("3YCojFePwaecuxQp1n391g%3D%3D")
client = PoeApi("3YCojFePwaecuxQp1n391g%3D%3D")
# client = PoeApi("p49wzQYujpnuLVWJ8eOKDw%3D%3D")

# Using Client with proxy (default is False)
# client = PoeApi("TOKEN_HERE", proxy=True)

**Mounted drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**get data**

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/content/drive/MyDrive/largeProject/code/QA/MRC.csv",index_col=False)

In [6]:
df.dropna(inplace=True)
list_context = set(df['context'].to_list())

In [15]:
df_context = pd.DataFrame({'context' : list(list_context)})

In [18]:
df_context.to_csv("/content/drive/MyDrive/largeProject/code/QA/context_gen.csv",index=None)

In [10]:
def get_prompt_new(context):
  prompt = f'''cho tôi 5 câu hỏi ngắn gọn dựa vào ngữ cảnh : "{context}"'''

  return prompt

In [11]:
# print(client.get_chat_history('chinchilla'))
dict_client = client.get_chat_history()

In [12]:
list_chat_code = []

for dict_id in dict_client['acouchy']:
  list_chat_code.append(dict_id['chatCode'])

KeyError: ignored

In [13]:
len(list_chat_code)

0

In [ ]:
dict_client

In [ ]:
import os
import random
from tqdm import tqdm
import time

In [ ]:
list_res = []
list_prompt = []
list_index = []

for index, row in tqdm(list_context):
    prompt =
    res = ""
    while True:
        try:
            for chunk in client.send_message("acouchy", prompt, chatCode = list_chat_code[index % len(list_chat_code)], timeout=30):
                pass
            res = chunk["text"]
            if res == "You are sending and receiving too many words in a short period of time.":
                continue
            break
        except Exception as e:
            print(e)
            if "Timed out waiting" in str(e):
                client = PoeApi("3YCojFePwaecuxQp1n391g%3D%3D")
                dict_chat = client.get_chat_history()["acouchy"]
                list_chat_code = [each['chatCode'] for each in dict_chat]
                time.sleep(10)
            res = ""
    res = " ".join(res.split("\n")).strip()

    list_res.append(res)
    list_prompt.append(prompt)
    list_index.append(index)

    if index % 50 == 0 and index != 0:
        time.sleep(10)
    if index % 100 == 0:
      dict_list = {
        "index" : list_index,
        "prompt" : list_prompt,
        "res" : list_res
      }

      if 'new_question.csv' not in os.listdir('/content/drive/MyDrive/largeProject'):
          df_test = pd.DataFrame.from_dict(dict_list)
          df_test.to_csv('/content/drive/MyDrive/largeProject/new_question.csv', index=False)
      else:
          df_test = pd.read_csv('/content/drive/MyDrive/largeProject/new_question.csv',index_col=None)
          df_test = pd.concat([df_test, pd.DataFrame.from_dict(dict_list)], ignore_index=True)
          df_test.to_csv('/content/drive/MyDrive/largeProject/new_question.csv', index=False)
    if index % 200 == 0 and index != 0:
        try:
          for idx_chat in list_chat_code:
              amount_previous_messages = len(client.get_previous_messages("acouchy", chatCode=idx_chat, get_all=True))
              if amount_previous_messages < 50:
                  continue
              client.purge_conversation("acouchy", chatCode=idx_chat, count=30)
          time.sleep(20)
        except Exception as e:
          print(e)

4405it [00:22, 94.42it/s] 


Retrying request 1/3 times...


4407it [01:20, 21.12it/s]


Retrying request 1/3 times...


4413it [02:50,  3.74it/s]

...


Retrying request 1/3 times...


4888it [3:41:31, 34.74s/it]


Retrying request 1/3 times...


4889it [3:42:11,  2.73s/it]


KeyboardInterrupt: ignored

In [ ]:
dict_list = {
    "index" : list_index,
    "prompt" : list_prompt,
    "res" : list_res
  }

df_test = pd.read_csv('/content/drive/MyDrive/largeProject/mrc_history1.csv',index_col=None)
df_test = pd.concat([df_test, pd.DataFrame.from_dict(dict_list)], ignore_index=True)
df_test.to_csv('/content/drive/MyDrive/largeProject/mrc_history1.csv', index=False)

In [ ]:
dict_list

{'index': [2165,
  2166,
  2167,
  2168,
  2169,
  2170,
  2171,
  2172,
  2173,
  2174,
  2175,
  2176,
  2177,
  2178,
  2179,
  2180,
  2181,
  2182,
  2183,
  2184,
  2185,
  2186,
  2187,
  2188,
  2189,
  2190,
  2191,
  2192,
  2193,
  2194,
  2195,
  2196,
  2197,
  2198,
  2199,
  2200],
 'prompt': ['Ngữ cảnh: "dựa vào các tháng gần nhất trong cùng 1 năm 2021 tính đến T7/2021, chỉ tiêu Dịch vụ ViettelPay của tổng công ty VDS được dự báo đạt được mốc KPI là 10.22% trong tháng kế tiếp. Kết quả này dự đoán sẽ được đánh giá không đạt so với mục tiêu KPI quý 3 có kết quả dự báo tháng tiếp theo là 10.22% và và so với mục tiêu KPI của năm đề ra là 12.8% được đánh giá là không đạt.". Câu hỏi "chỉ tiêu Dịch vụ ViettelPay được dự báo vào tháng kế tiếp có đạt KPI quý 3 không?". Trả lời câu hỏi trên theo từng bước dựa vào thông tin trong ngữ cảnh với tối đa 100 từ.',
  'Ngữ cảnh: "dựa vào các tháng gần nhất trong cùng 1 năm 2021 tính đến T7/2021, chỉ tiêu Dịch vụ ViettelPay của tổng công 

In [ ]:
text = "chỉ tiêu Tỷ lệ hài lòng của KH dịch vụ chuyển phát của tổng công ty VTPOST được đánh gía là không đạt chỉ tiêu hiện đã chỉ bằng 0.88 lần so với mục tiêu kpi ban đầu đề ra là 97.2%. Đối với kpi quý trước có được là 95.9%. Giảm 3.8% so với kết quả là với 2/2022. Câu hỏi kpi quý trước là bao nhiêu?"

print(text, end="", flush=True)

chỉ tiêu Tỷ lệ hài lòng của KH dịch vụ chuyển phát của tổng công ty VTPOST được đánh gía là không đạt chỉ tiêu hiện đã chỉ bằng 0.88 lần so với mục tiêu kpi ban đầu đề ra là 97.2%. Đối với kpi quý trước có được là 95.9%. Giảm 3.8% so với kết quả là với 2/2022. Câu hỏi kpi quý trước là bao nhiêu?